
Copyright **`(c)`** 2024 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [78]:
from collections import deque, namedtuple
from random import choice
from tqdm.auto import tqdm
import numpy as np
import heapq
import copy
import random


In [79]:
PUZZLE_DIM = 4
action = namedtuple('Action', ['pos1', 'pos2'])

In [80]:
### Determina le possibili mosse per la casella vuota.
def available_actions(state: np.ndarray) -> list['Action']:
    x, y = [int(_[0]) for _ in np.where(state == 0)] # find the position of the empty tile
    actions = list()

    # controlla le possibili direzioni in cui la casella vuota può muoversi (su, giù, sinistra, destra)
    # Se è possibile muoversi in quella direzione, aggiunge l'azione corrispondente alla lista actions.
    if x > 0:
        actions.append(action((x, y), (x - 1, y)))
    if x < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x + 1, y)))
    if y > 0:
        actions.append(action((x, y), (x, y - 1)))
    if y < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x, y + 1)))
    return actions


# scambia la posizione del pezzo vuoto con un'altra casella
def do_action(state: np.ndarray, action: 'Action') -> np.ndarray:
    new_state = state.copy()
    new_state[action.pos1], new_state[action.pos2] = new_state[action.pos2], new_state[action.pos1]
    return new_state

In [81]:
RANDOMIZE_STEPS = 10_000
state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
for r in tqdm(range(RANDOMIZE_STEPS), desc='Randomizing'):
    state = do_action(state, choice(available_actions(state)))
state

Randomizing:   0%|          | 0/10000 [00:00<?, ?it/s]

array([[ 7,  3,  2,  9],
       [10,  8,  4, 12],
       [ 1, 13,  0,  6],
       [ 5, 14, 11, 15]])

In [82]:
class PuzzleState:
    def __init__(self, board, empty_pos, g, prev=None):
        self.board = board
        self.empty_pos = empty_pos
        self.g = g  # Costo dal nodo 
        self.prev = prev
        self.h = self.calculate_heuristic()
        self.f = self.g + self.h

    def calculate_heuristic(self, alternative=False):
        # Distanza di Manhattan e Misplaced Tiles
        h_manhattan = 0
        misplaced_tiles = 0
        for i in range(len(self.board)):
            for j in range(len(self.board[i])):
                value = self.board[i][j]
                if value != 0:
                    target_x = (value - 1) // len(self.board)
                    target_y = (value - 1) % len(self.board)
                    if alternative:
                        # Alternativa: distanza di Manhattan ponderata
                        h_manhattan += 1.2 * (abs(target_x - i) + abs(target_y - j))
                    else:
                        h_manhattan += abs(target_x - i) + abs(target_y - j)
                    # Contare i misplaced tiles
                    if (i, j) != (target_x, target_y):
                        misplaced_tiles += 1

        # Combinare le due euristiche
        weight = 1  # Peso misplaced tiles
        h_combined = h_manhattan + weight * misplaced_tiles
        return h_combined


    def update_heuristic(self, alternative=False):
        self.h = self.calculate_heuristic(alternative)
        self.f = self.g + self.h

    def get_neighbors(self, alternative_heuristic=False):
        neighbors = []
        x, y = self.empty_pos
        directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]

        for dx, dy in directions:
            new_x, new_y = x + dx, y + dy
            if 0 <= new_x < len(self.board) and 0 <= new_y < len(self.board[0]):
                new_board = self.board.copy()
                new_board[x][y], new_board[new_x][new_y] = new_board[new_x][new_y], new_board[x][y]
                neighbor = PuzzleState(new_board, (new_x, new_y), self.g + 0.7, self)
                neighbor.update_heuristic(alternative_heuristic)
                neighbors.append(neighbor)
        return neighbors

    def is_goal(self):
        n = len(self.board)
        goal = np.array([[n * i + j + 1 for j in range(n)] for i in range(n)])
        goal[-1][-1] = 0
        return np.array_equal(self.board, goal)

    def __lt__(self, other):
        return self.f < other.f

def a_star(initial_board, max_iterations_without_improvement=7000):
    n = len(initial_board)
    empty_pos = None
    for i in range(n):
        for j in range(n):
            if initial_board[i][j] == 0:
                empty_pos = (i, j)
                break

    start_state = PuzzleState(initial_board, empty_pos, 0)
    open_list = []
    heapq.heappush(open_list, start_state)
    closed_set = set()
    actions_count = 0  
    iterations_without_improvement = 0
    previous_best_f = float('inf')
    use_alternative_heuristic = False

    while open_list:
        current_state = heapq.heappop(open_list)

        if current_state.is_goal():
            return reconstruct_path(current_state), actions_count

        closed_set.add(tuple(map(tuple, current_state.board)))

        # Controlla costo f è migliorato
        if current_state.f < previous_best_f:
            previous_best_f = current_state.f
            iterations_without_improvement = 0
        else:
            iterations_without_improvement += 1

        # Cambia l'euristica se non c'è miglioramento 
        if iterations_without_improvement >= max_iterations_without_improvement:
            use_alternative_heuristic = not use_alternative_heuristic
            iterations_without_improvement = 0

        for neighbor in current_state.get_neighbors(use_alternative_heuristic):
            actions_count += 1  # Incrementare contatore azioni
            if tuple(map(tuple, neighbor.board)) in closed_set:
                continue
            heapq.heappush(open_list, neighbor)

    return None, actions_count

def reconstruct_path(state):
    path = []
    while state:
        path.append(state.board)
        state = state.prev
    return path[::-1]

solution, actions_count = a_star(state)

if solution:
    print(f"Numero di step per arrivare alla soluzione: {len(solution) - 1}")  # quality 
    print(f"Numero di azioni fatte per arrivare alla soluzione: {actions_count}")  # cost 
    print("Soluzione trovata:")
    for row in solution[-1]:
        print(row)
else:
    print("Nessuna soluzione trovata.")

Numero di step per arrivare alla soluzione: 48
Numero di azioni fatte per arrivare alla soluzione: 28758
Soluzione trovata:
[1 2 3 4]
[5 6 7 8]
[ 9 10 11 12]
[13 14 15  0]
